In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
# read data
train_data_df = pd.read_csv('/kaggle/input/titanic/train.csv')

In [3]:
# check data
train_data_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# check shape
train_data_df.shape

(891, 12)

In [5]:
# check for null values
train_data_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
# impute age with median
train_data_df['Age'].fillna(train_data_df['Age'].median(),inplace=True)

In [7]:
# check for null values
train_data_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [8]:
# check cabin
train_data_df['Cabin'].value_counts

<bound method IndexOpsMixin.value_counts of 0       NaN
1       C85
2       NaN
3      C123
4       NaN
       ... 
886     NaN
887     B42
888     NaN
889    C148
890     NaN
Name: Cabin, Length: 891, dtype: object>

In [9]:
# drop cabin column
train_data_df.drop('Cabin',axis=1,inplace=True)

In [10]:
# drop name column, ticket column
train_data_df.drop(["Name","Ticket"],axis=1,inplace=True)

In [11]:
# check sex column
train_data_df.Sex.value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [12]:
# make male = 1, female = 0
train_data_df['Sex'] = train_data_df['Sex'].map({'male': 1, 'female': 0})

In [13]:
# check df
train_data_df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.0,1,0,7.2500,S
1,2,1,1,0,38.0,1,0,71.2833,C
2,3,1,3,0,26.0,0,0,7.9250,S
3,4,1,1,0,35.0,1,0,53.1000,S
4,5,0,3,1,35.0,0,0,8.0500,S


In [14]:
# convert age as int
train_data_df['Age'] = train_data_df['Age'].astype(int)

In [15]:
# check embarked column
train_data_df.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [16]:
# Assuming your dataframe is called df and the column to encode is called 'category'
encoded_df = pd.get_dummies(train_data_df['Embarked'], prefix='Embarked')

# Concatenate the original dataframe with the encoded columns
df_encoded = pd.concat([train_data_df, encoded_df], axis=1)

# Drop the original 'category' column if desired
df_encoded = df_encoded.drop('Embarked', axis=1)


In [17]:
# Splitting the data into features and target variable
X = df_encoded.drop('Survived', axis=1)  # Features
y = df_encoded['Survived']  # Target variable

In [18]:
from sklearn.model_selection import train_test_split

# Splitting the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [20]:
import tensorflow as tf

# Build the ANN model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [21]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
23/23 [==============================] - 1s 10ms/step - loss: 0.6130 - accuracy: 0.6713 - val_loss: 0.5198 - val_accuracy: 0.7598
Epoch 2/20
23/23 [==============================] - 0s 3ms/step - loss: 0.4974 - accuracy: 0.7851 - val_loss: 0.4551 - val_accuracy: 0.8101
Epoch 3/20
23/23 [==============================] - 0s 3ms/step - loss: 0.4500 - accuracy: 0.8202 - val_loss: 0.4335 - val_accuracy: 0.8101
Epoch 4/20
23/23 [==============================] - 0s 3ms/step - loss: 0.4287 - accuracy: 0.8062 - val_loss: 0.4272 - val_accuracy: 0.8045
Epoch 5/20
23/23 [==============================] - 0s 3ms/step - loss: 0.4152 - accuracy: 0.8174 - val_loss: 0.4209 - val_accuracy: 0.8101
Epoch 6/20
23/23 [==============================] - 0s 3ms/step - loss: 0.4071 - accuracy: 0.8244 - val_loss: 0.4145 - val_accuracy: 0.8101
Epoch 7/20
23/23 [==============================] - 0s 3ms/step - loss: 0.4038 - accuracy: 0.8357 - val_loss: 0.4178 - val_accuracy: 0.8156
Epoch 8/20
23/23 [=